In [54]:
import matplotlib.pyplot as plt
import cv2
from patchify import patchify, unpatchify
from skimage.filters import threshold_otsu, gaussian
import skimage.io as io
import os
from skimage import io
import numpy as np
from skimage.color import rgb2hed, hed2rgb
from skimage.morphology import binary_closing, binary_opening, disk
from skimage.segmentation import watershed
from skimage.feature import peak_local_max
from skimage.measure import label
from scipy import ndimage as ndi
from skimage.measure import label, regionprops

# Specify the directory containing the images
image_dir = 'Gloms'

# List all files in the directory
image_files = [f for f in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, f))]

# Initialize a list to store images
images = []
nuclei_each_glom = []
# Read each image
i=0
for file in image_files:
    i+=1
    file = 'P44F_PAS_structure1_image' + str(i) + '.png'
    # Construct full file path
    file_path = os.path.join(image_dir, file)
    # Read and append the image to the list
    img = io.imread(file_path)
    hed = rgb2hed(img)
    print(hed.shape)
    hematoxylin_channel = hed[:, :, 0]
    image_array = hematoxylin_channel
    # plt.imshow(image_array, cmap = 'viridis' )
    # plt.show()
    all_channels = image_array[:, :]  # Adjust depending on the specific dimensions/order
    Hemo = all_channels
    thresh_value = threshold_otsu((Hemo))
    binary_image = Hemo > thresh_value
    # Create a structuring element: You can adjust the size of the disk for your specific needs
    selem = disk(2)  # Adjust the size (2) according to your needs
    # Apply Closing operation
    closed_img = binary_closing(binary_image, selem)
    # Apply Opening operation
    opened_img = binary_opening(closed_img, selem)
    distance = ndi.distance_transform_edt(opened_img)
    markers = label(distance)
    labels = watershed(-distance, markers, mask=opened_img)
    # print(binary_image.shape)
    output_image = np.zeros(binary_image.shape, dtype=np.uint8)

    # Fill the regions corresponding to the nuclei with white
    for region in regionprops(labels):
        coords = region.coords
        output_image[coords[:, 0], coords[:, 1]] = 255  # Color the nuclei white
    print(np.unique(output_image))
    # Save the output image with detected nuclei
    output_path = 'Glom_masks/'+ 'P44F_PAS_structure1_image' + str(i) + '.png'
    cv2.imwrite(output_path, output_image)
    # Count the number of detected nuclei
    nuclei_count = len(regionprops(labels))
    # print(f'Number of nuclei detected: {nuclei_count}')
    print(i)
    # Calculate the area of each nucleus
    nuclei_areas = [region.area for region in regionprops(labels)]

    # Filter out nuclei with areas less than 10 pixels
    filtered_nuclei_areas = [area for area in nuclei_areas if area >= 1000]

    # Calculate the number of nuclei after thresholding
    thresholded_nuclei_count = len(filtered_nuclei_areas)
    print(f'Number of nuclei with area >= 40 pixels: {thresholded_nuclei_count}')
    nuclei_each_glom.append(thresholded_nuclei_count)
    if i == 50:
        break

(431, 471, 3)
[  0 255]
1
Number of nuclei with area >= 40 pixels: 80
(393, 351, 3)
[  0 255]
2
Number of nuclei with area >= 40 pixels: 34
(362, 345, 3)
[  0 255]
3
Number of nuclei with area >= 40 pixels: 22
(334, 353, 3)
[  0 255]
4
Number of nuclei with area >= 40 pixels: 18
(358, 334, 3)
[  0 255]
5
Number of nuclei with area >= 40 pixels: 17
(413, 398, 3)
[  0 255]
6
Number of nuclei with area >= 40 pixels: 49
(425, 446, 3)
[  0 255]
7
Number of nuclei with area >= 40 pixels: 67
(501, 477, 3)
[  0 255]
8
Number of nuclei with area >= 40 pixels: 90
(475, 502, 3)
[  0 255]
9
Number of nuclei with area >= 40 pixels: 90
(464, 519, 3)
[  0 255]
10
Number of nuclei with area >= 40 pixels: 97
(534, 546, 3)
[  0 255]
11
Number of nuclei with area >= 40 pixels: 101
(636, 487, 3)
[  0 255]
12
Number of nuclei with area >= 40 pixels: 101
(555, 477, 3)
[  0 255]
13
Number of nuclei with area >= 40 pixels: 97
(466, 415, 3)
[  0 255]
14
Number of nuclei with area >= 40 pixels: 70
(459, 462, 3)

In [55]:
# ! pip install openpyxl
sheet = pd.read_excel('Book1.xlsx')
manual = (sheet['Manual nuclei count'])
print(list(manual))

[32, 9, 6, 10, 8, 14, 17, 32, 25, 28, 37, 40, 32, 30, 31, 22, 33, 23, 21, 21, 27, 20, 15, 32, 24, 20, 33, 12, 30, 20, 22, 23, 21, 11, 21, 29, 26, 19, 33, 34, 27, 27, 25, 21, 20, 26, 15, 48, 29, 12]


In [56]:
from sklearn.metrics import r2_score

# Calculate R2 score
r2 = r2_score(manual, nuclei_each_glom)

print("R2 Score:", r2)


R2 Score: -22.719359104073316


In [43]:
# ! pip install pandas
import pandas as pd

# Specify the path to your Excel file
file_path = 'Book1.xlsx'

# Load the Excel file
df = pd.read_excel(file_path)

# Read the column "Segmented Output"
segmented_output = df['Automated nuclei count'].tolist()

# Assume this is the new list you want to add
# new_data = [1, 2, 3, 4, 5]  # Replace or modify this list as needed

# Add the list as a new column to the DataFrame
df['Automated nuclei count'] = pd.Series(nuclei_each_glom)

# Specify where to save the updated file
new_file_path = 'Book1.xlsx'

# Write the DataFrame back to Excel, not including the index
df.to_excel(new_file_path, index=False)


ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [39]:
import pandas as pd
file_path = 'Book1.xlsx'

# Load the Excel file
df = pd.read_excel(file_path)

# Create a DataFrame from the list
df_new = pd.DataFrame(nuclei_each_glom, columns=['Automated nuclei count'])

# Specify the path for the new Excel file
output_file_path = 'Book1.xlsx'

# Write the DataFrame to an Excel file, not including the index
df_new.to_excel(output_file_path, index=False)

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [ ]:
sheet = pd.read_excel(file_location)
print(sheet['Sl'])
